# Comparacion y conversion de modelos 3D (OBJ, STL, GLTF)


## 1) Instalacion de librerias


In [ ]:
!pip -q install trimesh open3d plotly pandas requests


## 2) Descarga de modelos (Suzanne en distintos formatos)

Fuente: `EverseDevelopment/3DModelsFamousSamples`


In [ ]:
from pathlib import Path
import requests

BASE_DIR = Path('models')
BASE_DIR.mkdir(exist_ok=True)

MODELS = {
    'suzanne': {
        'obj': 'https://s3.amazonaws.com/everse.assets/common3dmodels/OBJ/suzanne.obj',
        'gltf': 'https://s3.amazonaws.com/everse.assets/common3dmodels/GLTF/suzanne.gltf',
        'stl': 'https://s3.amazonaws.com/everse.assets/common3dmodels/STL/suzanne.stl',
    }
}

def download_file(url: str, dst: Path, timeout=60):
    if dst.exists() and dst.stat().st_size > 0:
        return dst
    dst.parent.mkdir(parents=True, exist_ok=True)
    r = requests.get(url, timeout=timeout)
    r.raise_for_status()
    with open(dst, 'wb') as f:
        f.write(r.content)
    return dst

downloaded = {}
for model_name, formats in MODELS.items():
    downloaded[model_name] = {}
    for ext, url in formats.items():
        out = BASE_DIR / model_name / f'{model_name}.{ext}'
        try:
            downloaded[model_name][ext] = download_file(url, out)
            print(f'[OK] {model_name}.{ext}')
        except Exception as e:
            print(f'[ERROR] {model_name}.{ext}: {e}')

downloaded


: 

## 3) Carga con trimesh + lectura de normales con open3d


In [ ]:
import numpy as np
import pandas as pd
import trimesh

def load_as_trimesh(path: Path):
    loaded = trimesh.load(path, force='mesh', process=False)
    if isinstance(loaded, trimesh.Scene):
        if len(loaded.geometry) == 0:
            raise ValueError(f'Escena vacia: {path}')
        loaded = loaded.dump(concatenate=True)
    return loaded

def duplicates_count_rows(arr: np.ndarray):
    if arr is None or len(arr) == 0:
        return 0
    unique = np.unique(arr, axis=0)
    return int(len(arr) - len(unique))

def faces_sorted(faces: np.ndarray):
    if faces is None or len(faces) == 0:
        return faces
    return np.sort(faces, axis=1)

records = []
meshes = {}

for model_name, formats in downloaded.items():
    meshes[model_name] = {}
    for ext, path in formats.items():
        if not Path(path).exists():
            continue
        try:
            mesh = load_as_trimesh(Path(path))
            meshes[model_name][ext] = mesh

            verts = np.asarray(mesh.vertices)
            faces = np.asarray(mesh.faces)
            v_dups = duplicates_count_rows(np.round(verts, 7))
            f_dups = duplicates_count_rows(faces_sorted(faces))

            records.append({
                'model': model_name,
                'format': ext,
                'path': str(path),
                'vertices': int(len(verts)),
                'faces': int(len(faces)),
                'vertex_normals_count_trimesh': int(len(mesh.vertex_normals)) if len(verts) else 0,
                'face_normals_count_trimesh': int(len(mesh.face_normals)) if len(faces) else 0,
                'duplicate_vertices': v_dups,
                'duplicate_faces': f_dups,
            })
        except Exception as e:
            records.append({
                'model': model_name,
                'format': ext,
                'path': str(path),
                'error': str(e),
            })

df = pd.DataFrame(records)
columnas_df = {
    'model': 'modelo',
    'format': 'formato',
    'path': 'ruta_archivo',
    'vertices': 'num_vertices',
    'faces': 'num_caras',
    'vertex_normals_count_trimesh': 'normales_vertice_trimesh',
    'face_normals_count_trimesh': 'normales_cara_trimesh',
    'duplicate_vertices': 'vertices_duplicados',
    'duplicate_faces': 'caras_duplicadas',
    'error': 'error_carga',
}
df = df.rename(columns=columnas_df)
df


## 4) Visualizacion de la figura por formato

Se usa Plotly para mostrar cada version en Colab.


In [ ]:
import plotly.graph_objects as go

def show_mesh_plotly(mesh, title='mesh', max_faces=200000):
    v = np.asarray(mesh.vertices)
    f = np.asarray(mesh.faces)

    # Limitar caras para no bloquear el navegador en Colab
    if len(f) > max_faces:
        idx = np.random.choice(len(f), size=max_faces, replace=False)
        f = f[idx]

    fig = go.Figure(data=[go.Mesh3d(
        x=v[:, 0], y=v[:, 1], z=v[:, 2],
        i=f[:, 0], j=f[:, 1], k=f[:, 2],
        color='lightblue',
        opacity=1.0,
        flatshading=False
    )])
    fig.update_layout(
        title=title,
        scene_aspectmode='data',
        width=800,
        height=600
    )
    fig.show()

for model_name, formats in meshes.items():
    for ext, mesh in formats.items():
        print(f'Visualizando: {model_name}.{ext}')
        show_mesh_plotly(mesh, title=f'{model_name}.{ext}')


## 5) Conversion con trimesh (un metodo: OBJ -> STL)


In [ ]:
OUT_CONV = Path('converted')
OUT_CONV.mkdir(exist_ok=True)

# Ejemplo: convertir suzanne.obj a suzanne_from_obj.stl
src = downloaded.get('suzanne', {}).get('obj', None)
if src and Path(src).exists():
    mesh = load_as_trimesh(Path(src))
    out_stl = OUT_CONV / 'suzanne_from_obj.stl'

    mesh.export(out_stl)

    print('Generado:')
    print('-', out_stl)
else:
    print('No se encontro suzanne.obj para conversion de ejemplo.')


## 6) Bonus: script automatizado de comparacion de archivos

Recorre una carpeta y compara archivos `obj/stl/gltf/glb`, guardando los resultados en un CSV.


In [ ]:
VALID_EXT = {'.obj', '.stl', '.gltf', '.glb'}

def compare_folder(folder: Path, out_csv: Path = Path('comparison_report.csv')):
    rows = []
    for p in sorted(folder.rglob('*')):
        if p.suffix.lower() not in VALID_EXT:
            continue
        try:
            m = load_as_trimesh(p)
            v = np.asarray(m.vertices)
            f = np.asarray(m.faces)

            rows.append({
                'file': str(p),
                'ext': p.suffix.lower(),
                'vertices': int(len(v)),
                'faces': int(len(f)),
                'dup_vertices': duplicates_count_rows(np.round(v, 7)),
                'dup_faces': duplicates_count_rows(faces_sorted(f)),
                'bbox_size': float(np.linalg.norm(m.bounds[1] - m.bounds[0])) if m.bounds is not None else None,
            })
        except Exception as e:
            rows.append({'file': str(p), 'ext': p.suffix.lower(), 'error': str(e)})

    report = pd.DataFrame(rows)
    columnas_report = {
        'file': 'archivo',
        'ext': 'formato',
        'vertices': 'num_vertices',
        'faces': 'num_caras',
        'dup_vertices': 'vertices_duplicados',
        'dup_faces': 'caras_duplicadas',
        'bbox_size': 'tamano_bbox',
        'error': 'error_carga',
    }
    report = report.rename(columns=columnas_report)
    report.to_csv(out_csv, index=False)
    return report

report = compare_folder(Path('models'))
report


## 7) Referencias de modelos

- EverseDevelopment/3DModelsFamousSamples: https://github.com/EverseDevelopment/3DModelsFamousSamples
